In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime
from mysql.connector import errors



## =================================
## CONFIGURATION

In [2]:
mysql_config = {
    "host": "localhost",
    "user": "root",
    "password": "@michreano",         
    "database": "nepaldb"
}

folder_path = r"C:\Users\micha\Documents\Portfolio_proj\Data"

## =========================
### CONNECT TO MySQL DATABASE.

In [3]:
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

## ======================
### FUNCTION TO DETECT COLUMN TYPES

In [ ]:
def infer_mysql_type(series: pd.Series) -> str:
    """Infer MySQL column type form pandas Series"""
    col_name = series.name.lower()
    # Drop NaN values for type inference
    sample = series.dropna()
    
    if sample.empty:
        return "VARCHAR(255)"
    
    # Try to detect integer
    if pd.api.types.is_integer_dtype(sample):
        # If maximum value is too large for INT, use BIGINT
        # Use sample.max() and sample.min() instead of series to avoid NaN issues
        if sample.max() > 2147483647 or sample.min() < -2147483648:
            return "BIGINT"
        else:
            return "INT"
    
    # try to detect float
    elif pd.api.types.is_float_dtype(sample):
        return "FLOAT"
    
    # try to detect datetime
    elif pd.api.types.is_datetime64_any_dtype(sample):
        return "DATETIME"
    
    # Attempt to parse date-like strings
    else:
    # Otherwise use TEXT
         if "_id" in col_name:
            return "VARCHAR(255)"
    
    # Default fallback
    return "TEXT"

## ==============================
### LOOPING THROUGH THE CSV OR THE EXCEL FILES.

In [ ]:
for file in os.listdir(folder_path):
    if file.endswith(".csv") or file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        table_name = os.path.splitext(file)[0].replace(" ", "_").lower()

        print(f"\n Importing {file} → Table: {table_name}")

        # --- Read file based on extension ---
        if file.endswith(".csv"):
            df = pd.read_csv(file_path, low_memory=False)
        else:
            df = pd.read_excel(file_path)

        # --- Clean column names ---
        df.columns = [col.strip().replace(" ", "_") for col in df.columns]
        df = df.replace({np.nan: None})

        # --- Build CREATE TABLE statement with inferred types ---
        columns_definitions = []
        for col in df.columns:
            mysql_type = infer_mysql_type(df[col])
            columns_definitions.append(f"`{col}` {mysql_type}")
        columns_query = ", ".join(columns_definitions)
        create_table = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_query});"

        cursor.execute(create_table)
        conn.commit()
        print(f" Table `{table_name}` created or already exists.")

        # --- Prepare INSERT statement ---
        cols = ", ".join([f"`{col}`" for col in df.columns])
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"

        # --- Insert data in manageable batches ---
        try:
            data = df.values.tolist()
            batch_size = 1000  # Adjust if necessary

            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]
                cursor.executemany(insert_query, batch)
                conn.commit()

            print(f" {file} imported successfully with {len(df)} rows.")

        except mysql.connector.errors.OperationalError as e:
            print(f" MySQL connection lost during {file}: {e}")
            conn.reconnect(attempts=3, delay=2)
            cursor = conn.cursor()
        except Exception as e:
            print(f" Error inserting data for {file}: {e}")

# --- Close connection ---
cursor.close()
conn.close()
print("\n All files imported into MySQL successfully — one table per file!")